Some preprocessing

In [1]:
RUN_PYTHON_SCRIPT = False

False

In [2]:
import sys
sys.path.append("../mypkg")
from constants import RES_ROOT, FIG_ROOT, DATA_ROOT

In [3]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
from easydict import EasyDict as edict
from tqdm import trange, tqdm
import time

if not RUN_PYTHON_SCRIPT:
    plt.style.use(FIG_ROOT/"base.mplstyle")

In [4]:
from spectrome import Brain
from sgm.sgm import SGM
from utils.misc import save_pkl, save_pkl_dict2folder, load_pkl, load_pkl_folder2dict, delta_time
from utils.standardize import std_mat, std_vec

In [5]:
# pkgs for pytorch ( Mar 27, 2023) 
import torch
import torch.nn as nn
from torch.functional import F
from torch.optim.lr_scheduler import ExponentialLR

torch.set_default_dtype(torch.float64)
if torch.cuda.is_available():
    torch.cuda.set_device(2)
    torch.set_default_tensor_type(torch.cuda.DoubleTensor)
    torch.backends.cudnn.benchmark = True
else:
    torch.set_default_tensor_type(torch.DoubleTensor)

# Get the spectrogram 


In [6]:
from scipy.io import loadmat
from scipy import signal
root = DATA_ROOT/"MEG-eye-multiFreqs"

_paras = edict()
_paras.freq_limit = [1, 45]
_paras.fs = 600
_paras.fils_path = list(root.glob("*1-45*20.mat"))

_paras.spg_paras = edict()
_paras.spg_paras.nperseg = 600
_paras.spg_paras.noverlap = 302

In [11]:
# change the file to estimate PSD
fil_path = _paras.fils_path[1]

MEG_ts = loadmat(fil_path)["DK_timecourse"];
PSD_data = edict()
PSD_data.spectrogram = []
PSD_data.freqs = None
PSD_data.times = None
PSD_data.paras = _paras.spg_paras
PSD_data.readme = "The spectrogram is estimated with signal.spectrogram with paras. Note that it is in |mag|^2 (not in dB)"
for roi_ix in trange(MEG_ts.shape[0]):
    freqs, times, Sxx = signal.spectrogram(MEG_ts[roi_ix], _paras.fs, mode="psd", 
                                           nperseg=_paras.spg_paras.nperseg, 
                                           noverlap=_paras.spg_paras.noverlap)
    keep_idxs = np.bitwise_and(freqs>=_paras.freq_limit[0],  freqs<=_paras.freq_limit[1])
    freqs = freqs[keep_idxs]
    Sxx = Sxx[keep_idxs]
    PSD_data.spectrogram.append(Sxx)
    
PSD_data.freqs = freqs
PSD_data.times = times
PSD_data.spectrogram = np.array(PSD_data.spectrogram)

save_pkl(root/(fil_path.stem + "_spectrogram.pkl"), PSD_data)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 68/68 [00:00<00:00, 486.42it/s]


Save to /data/rajlab1/user_data/jin/MyResearch/TV-SGM/notebooks/../mypkg/../data/MEG-eye-multiFreqs/lcmv_DKregion_R2868E_03-st_1-45Hz_20_spectrogram.pkl


In [88]:
#plt.pcolormesh(times, freqs, 10 * np.log10(Sxx), shading='gouraud')
#plt.ylabel('Frequency [Hz]')
#plt.xlabel('Time [sec]')
#plt.colorbar(label='PSD (dB)')
#plt.show()

(240,)

# produce base metrics

In [7]:
from models.loss import reg_R_fn, lin_R_fn

In [8]:
import netCDF4
fils = list(DATA_ROOT.glob("*s100tp.nc"))
file2read = netCDF4.Dataset(fils[0], 'r')
psd_all = np.array(file2read.variables["__xarray_dataarray_variable__"][:])
psd_all = 10 * np.log10(psd_all)
time_points = np.array(file2read.variables["timepoints"][:])
freqs = np.array(file2read.variables["frequencies"][:])
ROIs_order = np.array(file2read.variables["regionx"][:])
file2read.close()

# real data
all_data = psd_all.transpose(3, 2, 0, 1)
all_data_std = (all_data - all_data.mean(axis=-1, keepdims=True))/all_data.std(axis=-1, keepdims=True);

In [9]:
# the simulated data
simu_sgm_data = load_pkl_folder2dict(RES_ROOT/"simu_sgm_data_ind_large", including=["PSDs_test"]);

Load file /data/rajlab1/user_data/jin/MyResearch/TV-SGM/notebooks/../mypkg/../results/simu_sgm_data_ind_large/PSDs_test.pkl


In [10]:
base_PSDs_dB = 20*np.log10(simu_sgm_data.PSDs_test[:1000]);
base_PSDs_dB_std = (base_PSDs_dB - base_PSDs_dB.mean(axis=-1, keepdims=True))/base_PSDs_dB.std(axis=-1, keepdims=True);

In [11]:
# path to save
target_path = DATA_ROOT/(fils[0].stem + "_base_1000")
if not target_path.exists():
    target_path.mkdir()

In [12]:
def _all2stat_fn(corrs):
    """output useful stats for the base corrs
    """
    res = edict()
    if corrs.ndim == 1:
        res.mean = np.mean(corrs)
        res.median = np.median(corrs)
        res.std = np.std(corrs)
    elif corrs.ndim == 2:
        assert corrs.shape[-1] == 68
        res.mean = np.mean(corrs, axis=0)
        res.median = np.median(corrs, axis=0)
        res.std = np.std(corrs, axis=0)
    return res

## Reg R

In [13]:
for sub_ix in trange(all_data_std.shape[0]):
    cur_sub_corrs = []
    cur_sub_corrs_roiwise = []
    for time_ix in range(all_data_std.shape[1]):
        cur_data = all_data_std[sub_ix, time_ix];
        cur_data_rep = np.repeat(cur_data[np.newaxis], len(base_PSDs_dB_std), axis=0)
        base_corrs_roiwise  = reg_R_fn(base_PSDs_dB_std, cur_data_rep)
        base_corrs = reg_R_fn(base_PSDs_dB_std.mean(axis=1), 
                              cur_data_rep.mean(axis=1))
        cur_sub_corrs_roiwise.append(_all2stat_fn(base_corrs_roiwise))
        cur_sub_corrs.append(_all2stat_fn(base_corrs))
        
    save_pkl(target_path/f"regR_{sub_ix}.pkl", cur_sub_corrs)
    save_pkl(target_path/f"regR_roiwise_{sub_ix}.pkl", cur_sub_corrs_roiwise)

  3%|███▊                                                                                                                                       | 1/36 [00:18<10:31, 18.03s/it]

/data/rajlab1/user_data/jin/MyResearch/TV-SGM/notebooks/../mypkg/../data/individual_wavelet_reordered_smooth_raw_s100tp_base_1000/regR_0.pkl exists! Use is_force=True to save it anyway
/data/rajlab1/user_data/jin/MyResearch/TV-SGM/notebooks/../mypkg/../data/individual_wavelet_reordered_smooth_raw_s100tp_base_1000/regR_roiwise_0.pkl exists! Use is_force=True to save it anyway


  6%|███████▋                                                                                                                                   | 2/36 [00:36<10:15, 18.11s/it]

Save to /data/rajlab1/user_data/jin/MyResearch/TV-SGM/notebooks/../mypkg/../data/individual_wavelet_reordered_smooth_raw_s100tp_base_1000/regR_1.pkl
Save to /data/rajlab1/user_data/jin/MyResearch/TV-SGM/notebooks/../mypkg/../data/individual_wavelet_reordered_smooth_raw_s100tp_base_1000/regR_roiwise_1.pkl


  8%|███████████▌                                                                                                                               | 3/36 [00:54<09:57, 18.10s/it]

...

Save to /data/rajlab1/user_data/jin/MyResearch/TV-SGM/notebooks/../mypkg/../data/individual_wavelet_reordered_smooth_raw_s100tp_base_1000/regR_33.pkl
Save to /data/rajlab1/user_data/jin/MyResearch/TV-SGM/notebooks/../mypkg/../data/individual_wavelet_reordered_smooth_raw_s100tp_base_1000/regR_roiwise_33.pkl


 97%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏   | 35/36 [10:32<00:18, 18.33s/it]

Save to /data/rajlab1/user_data/jin/MyResearch/TV-SGM/notebooks/../mypkg/../data/individual_wavelet_reordered_smooth_raw_s100tp_base_1000/regR_34.pkl
Save to /data/rajlab1/user_data/jin/MyResearch/TV-SGM/notebooks/../mypkg/../data/individual_wavelet_reordered_smooth_raw_s100tp_base_1000/regR_roiwise_34.pkl


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 36/36 [10:50<00:00, 18.08s/it]

Save to /data/rajlab1/user_data/jin/MyResearch/TV-SGM/notebooks/../mypkg/../data/individual_wavelet_reordered_smooth_raw_s100tp_base_1000/regR_35.pkl
Save to /data/rajlab1/user_data/jin/MyResearch/TV-SGM/notebooks/../mypkg/../data/individual_wavelet_reordered_smooth_raw_s100tp_base_1000/regR_roiwise_35.pkl


## linR

In [ ]:
for sub_ix in trange(all_data_std.shape[0]):
    cur_sub_corrs = []
    cur_sub_corrs_roiwise = []
    for time_ix in range(all_data_std.shape[1]):
        cur_data = all_data_std[sub_ix, time_ix];
        cur_data_rep = np.repeat(cur_data[np.newaxis], len(base_PSDs_dB_std), axis=0)
        base_corrs_roiwise  = lin_R_fn(base_PSDs_dB_std, cur_data_rep)
        base_corrs = lin_R_fn(base_PSDs_dB_std.mean(axis=1), 
                              cur_data_rep.mean(axis=1))
        cur_sub_corrs_roiwise.append(_all2stat_fn(base_corrs_roiwise))
        cur_sub_corrs.append(_all2stat_fn(base_corrs))
        
    save_pkl(target_path/f"linR_{sub_ix}.pkl", cur_sub_corrs)
    save_pkl(target_path/f"linR_roiwise_{sub_ix}.pkl", cur_sub_corrs_roiwise)

  0%|                                                                                                                                                   | 0/36 [00:00<?, ?it/s]